In [1]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,LSTM,Dropout,Embedding
import nltk
nltk.download('gutenberg')
from nltk.corpus import gutenberg
data=gutenberg.raw('shakespeare-hamlet.txt')
with open('hamlet.txt','w') as file:
    file.write(data)

[nltk_data] Error loading gutenberg: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>


### Data preprocessing

In [2]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

In [3]:
with open('hamlet.txt','r') as file:
    text=file.read().lower()
#tokenize    
token=Tokenizer()
token.fit_on_texts([text])
total_words=len(token.word_index)+1

In [4]:
total_words

4818

In [5]:
token.word_index

{'the': 1,
 'and': 2,
 'to': 3,
 'of': 4,
 'i': 5,
 'you': 6,
 'a': 7,
 'my': 8,
 'it': 9,
 'in': 10,
 'that': 11,
 'ham': 12,
 'is': 13,
 'not': 14,
 'his': 15,
 'this': 16,
 'with': 17,
 'your': 18,
 'but': 19,
 'for': 20,
 'me': 21,
 'lord': 22,
 'as': 23,
 'what': 24,
 'he': 25,
 'be': 26,
 'so': 27,
 'him': 28,
 'haue': 29,
 'king': 30,
 'will': 31,
 'no': 32,
 'our': 33,
 'we': 34,
 'on': 35,
 'are': 36,
 'if': 37,
 'all': 38,
 'then': 39,
 'shall': 40,
 'by': 41,
 'thou': 42,
 'come': 43,
 'or': 44,
 'hamlet': 45,
 'good': 46,
 'do': 47,
 'hor': 48,
 'her': 49,
 'let': 50,
 'now': 51,
 'thy': 52,
 'how': 53,
 'more': 54,
 'they': 55,
 'from': 56,
 'enter': 57,
 'at': 58,
 'was': 59,
 'oh': 60,
 'like': 61,
 'most': 62,
 'there': 63,
 'well': 64,
 'know': 65,
 'selfe': 66,
 'would': 67,
 'them': 68,
 'loue': 69,
 'may': 70,
 "'tis": 71,
 'vs': 72,
 'sir': 73,
 'qu': 74,
 'which': 75,
 'did': 76,
 'why': 77,
 'laer': 78,
 'giue': 79,
 'thee': 80,
 'ile': 81,
 'must': 82,
 'hath': 

In [6]:
token.word_index.items()

dict_items([('the', 1), ('and', 2), ('to', 3), ('of', 4), ('i', 5), ('you', 6), ('a', 7), ('my', 8), ('it', 9), ('in', 10), ('that', 11), ('ham', 12), ('is', 13), ('not', 14), ('his', 15), ('this', 16), ('with', 17), ('your', 18), ('but', 19), ('for', 20), ('me', 21), ('lord', 22), ('as', 23), ('what', 24), ('he', 25), ('be', 26), ('so', 27), ('him', 28), ('haue', 29), ('king', 30), ('will', 31), ('no', 32), ('our', 33), ('we', 34), ('on', 35), ('are', 36), ('if', 37), ('all', 38), ('then', 39), ('shall', 40), ('by', 41), ('thou', 42), ('come', 43), ('or', 44), ('hamlet', 45), ('good', 46), ('do', 47), ('hor', 48), ('her', 49), ('let', 50), ('now', 51), ('thy', 52), ('how', 53), ('more', 54), ('they', 55), ('from', 56), ('enter', 57), ('at', 58), ('was', 59), ('oh', 60), ('like', 61), ('most', 62), ('there', 63), ('well', 64), ('know', 65), ('selfe', 66), ('would', 67), ('them', 68), ('loue', 69), ('may', 70), ("'tis", 71), ('vs', 72), ('sir', 73), ('qu', 74), ('which', 75), ('did', 76

In [7]:
text

"[the tragedie of hamlet by william shakespeare 1599]\n\n\nactus primus. scoena prima.\n\nenter barnardo and francisco two centinels.\n\n  barnardo. who's there?\n  fran. nay answer me: stand & vnfold\nyour selfe\n\n   bar. long liue the king\n\n   fran. barnardo?\n  bar. he\n\n   fran. you come most carefully vpon your houre\n\n   bar. 'tis now strook twelue, get thee to bed francisco\n\n   fran. for this releefe much thankes: 'tis bitter cold,\nand i am sicke at heart\n\n   barn. haue you had quiet guard?\n  fran. not a mouse stirring\n\n   barn. well, goodnight. if you do meet horatio and\nmarcellus, the riuals of my watch, bid them make hast.\nenter horatio and marcellus.\n\n  fran. i thinke i heare them. stand: who's there?\n  hor. friends to this ground\n\n   mar. and leige-men to the dane\n\n   fran. giue you good night\n\n   mar. o farwel honest soldier, who hath relieu'd you?\n  fra. barnardo ha's my place: giue you goodnight.\n\nexit fran.\n\n  mar. holla barnardo\n\n   bar. 

In [8]:
# create input sequence
inputsequence=[]
for line in text.split('\n'):#split by each line
    token_list=token.texts_to_sequences([line])[0] #each line text is converted into sequence of token (based on index)
    for i in range(1,len(token_list)):
        n_gram_seq=token_list[:i+1] # generate multiple subsets of token_list
        inputsequence.append(n_gram_seq)

In [9]:
inputsequence

[[1, 687],
 [1, 687, 4],
 [1, 687, 4, 45],
 [1, 687, 4, 45, 41],
 [1, 687, 4, 45, 41, 1886],
 [1, 687, 4, 45, 41, 1886, 1887],
 [1, 687, 4, 45, 41, 1886, 1887, 1888],
 [1180, 1889],
 [1180, 1889, 1890],
 [1180, 1889, 1890, 1891],
 [57, 407],
 [57, 407, 2],
 [57, 407, 2, 1181],
 [57, 407, 2, 1181, 177],
 [57, 407, 2, 1181, 177, 1892],
 [407, 1182],
 [407, 1182, 63],
 [408, 162],
 [408, 162, 377],
 [408, 162, 377, 21],
 [408, 162, 377, 21, 247],
 [408, 162, 377, 21, 247, 882],
 [18, 66],
 [451, 224],
 [451, 224, 248],
 [451, 224, 248, 1],
 [451, 224, 248, 1, 30],
 [408, 407],
 [451, 25],
 [408, 6],
 [408, 6, 43],
 [408, 6, 43, 62],
 [408, 6, 43, 62, 1893],
 [408, 6, 43, 62, 1893, 96],
 [408, 6, 43, 62, 1893, 96, 18],
 [408, 6, 43, 62, 1893, 96, 18, 566],
 [451, 71],
 [451, 71, 51],
 [451, 71, 51, 1894],
 [451, 71, 51, 1894, 567],
 [451, 71, 51, 1894, 567, 378],
 [451, 71, 51, 1894, 567, 378, 80],
 [451, 71, 51, 1894, 567, 378, 80, 3],
 [451, 71, 51, 1894, 567, 378, 80, 3, 273],
 [451, 71

In [10]:
token.word_index['tragedie']

687

In [11]:
for line in text.split('\n'):
    token_list=token.texts_to_sequences([line])[0]
    print(token_list)

[1, 687, 4, 45, 41, 1886, 1887, 1888]
[]
[]
[1180, 1889, 1890, 1891]
[]
[57, 407, 2, 1181, 177, 1892]
[]
[407, 1182, 63]
[408, 162, 377, 21, 247, 882]
[18, 66]
[]
[451, 224, 248, 1, 30]
[]
[408, 407]
[451, 25]
[]
[408, 6, 43, 62, 1893, 96, 18, 566]
[]
[451, 71, 51, 1894, 567, 378, 80, 3, 273, 1181]
[]
[408, 20, 16, 1895, 114, 379, 71, 883, 491]
[2, 5, 92, 688, 58, 144]
[]
[346, 29, 6, 108, 568, 884]
[408, 14, 7, 885, 1896]
[]
[346, 64, 380, 37, 6, 47, 689, 120, 2]
[347, 1, 1897, 4, 8, 257, 492, 68, 87, 149]
[57, 120, 2, 347]
[]
[408, 5, 117, 5, 139, 68, 247, 1182, 63]
[48, 205, 3, 16, 409]
[]
[140, 2, 1898, 348, 3, 1, 493]
[]
[408, 79, 6, 46, 124]
[]
[140, 125, 1899, 322, 1183, 121, 83, 1900, 6]
[1901, 407, 258, 8, 494, 79, 6, 380]
[]
[218, 408]
[]
[140, 1902, 407]
[]
[451, 94, 24, 13, 120, 63]
[48, 7, 495, 4, 28]
[]
[451, 236, 120, 236, 46, 347]
[]
[140, 24, 258, 16, 167, 886, 131, 3, 124]
[]
[451, 5, 29, 189, 153]
[]
[140, 120, 887, 71, 19, 33, 1184]
[2, 31, 14, 50, 1903, 136, 168, 4

### Padding

In [12]:
max([len(x) for x in inputsequence])

14

In [13]:
# padding
padded=np.array(pad_sequences(inputsequence,maxlen=14,padding='pre'))

In [14]:
padded

array([[   0,    0,    0, ...,    0,    1,  687],
       [   0,    0,    0, ...,    1,  687,    4],
       [   0,    0,    0, ...,  687,    4,   45],
       ...,
       [   0,    0,    0, ...,    4,   45, 1047],
       [   0,    0,    0, ...,   45, 1047,    4],
       [   0,    0,    0, ..., 1047,    4,  193]])

In [15]:
padded[0]

array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   1,
       687])

In [16]:
x=padded[:,:-1] # remove final words of each line
x

array([[   0,    0,    0, ...,    0,    0,    1],
       [   0,    0,    0, ...,    0,    1,  687],
       [   0,    0,    0, ...,    1,  687,    4],
       ...,
       [   0,    0,    0, ...,  687,    4,   45],
       [   0,    0,    0, ...,    4,   45, 1047],
       [   0,    0,    0, ...,   45, 1047,    4]])

In [17]:
y=padded[:,-1:]#take last word only
y

array([[ 687],
       [   4],
       [  45],
       ...,
       [1047],
       [   4],
       [ 193]])

In [18]:
import tensorflow as tf

In [19]:
y=tf.keras.utils.to_categorical(y,num_classes=total_words) #one hot encoding
y

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

### Train Test Split

In [20]:
x_train,x_test,y_train,y_test=train_test_split(x,y,train_size=0.8)

In [21]:
x_train.shape

(20585, 13)

In [22]:
y_train.shape

(20585, 4818)

In [23]:
x_test.shape

(5147, 13)

In [24]:
y_test.shape

(5147, 4818)

### Model Building

In [25]:
#pip install --upgrade tensorflow keras


In [26]:
model=Sequential()
model.add(Embedding(total_words,100,input_length=14))
model.add(LSTM(150,return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(100))
model.add(Dense(total_words,activation='softmax'))

C:\Users\aksha\anaconda3\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [27]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [28]:
model.build(input_shape=(None, 14))
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 14, 100)        │       481,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 14, 150)        │       150,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 14, 150)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 100)            │       100,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 4818)           │       486,618 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,219,418 (4.65 MB)

 Trainable params: 1,219,418 (4.65 MB)

 Non-trainable params: 0 (0.00 B)

In [51]:
history=model.fit(x_train,y_train,epochs=50,validation_data=(x_test,y_test),verbose=1)

Epoch 1/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 22s 35ms/step - accuracy: 0.7167 - loss: 1.2124 - val_accuracy: 0.0513 - val_loss: 14.2872
Epoch 2/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 22s 35ms/step - accuracy: 0.7255 - loss: 1.1845 - val_accuracy: 0.0490 - val_loss: 14.2568
Epoch 3/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 40s 32ms/step - accuracy: 0.7276 - loss: 1.1820 - val_accuracy: 0.0528 - val_loss: 14.3299
Epoch 4/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 41s 32ms/step - accuracy: 0.7268 - loss: 1.1775 - val_accuracy: 0.0497 - val_loss: 14.3216
Epoch 5/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 22s 34ms/step - accuracy: 0.7312 - loss: 1.1499 - val_accuracy: 0.0517 - val_loss: 14.4077
Epoch 6/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 40s 33ms/step - accuracy: 0.7376 - loss: 1.1430 - val_accuracy: 0.0509 - val_loss: 14.4110
Epoch 7/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 22s 34ms/step - accuracy: 0.7289 - loss: 1.1503 - val_accuracy: 0.0499 - val_loss: 14.4730
Epoch 8/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 22s 34ms/step - accuracy: 0.7326 - loss: 1.

644/644 ━━━━━━━━━━━━━━━━━━━━ 22s 34ms/step - accuracy: 0.7867 - loss: 0.8803 - val_accuracy: 0.0490 - val_loss: 15.7809


In [52]:
def predict_next(model,tokenizer,text,max_sequence_len):
    token_list=tokenizer.texts_to_sequences([text])[0]
    if len(token_list)>=max_sequence_len:
        token_list=token_list[-(max_sequence_len-1):] #taking a subset of words if token_list length is greater than max_sequence_len 
        #token_list = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
        #max_sequence_len = 5 ,token_list[-(max_sequence_len-1):] will be token_list[-4:], which gives [7, 8, 9, 10].
    else:
        token_list = token_list
        
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
    
    predicted = model.predict(token_list, verbose=0)
    predict_word_index = np.argmax(predicted, axis=1)[0]  # Extract the integer from the array
    
    for word, index in tokenizer.word_index.items():
        if index == predict_word_index:
            return word
    
    return None

In [53]:
text1='To be or not to be'
max_seq=model.input_shape[1]+1
next_word=predict_next(model,token,text1,max_seq)

In [54]:
next_word

"'tis"

In [55]:
text1='To be or not to be not'
max_seq=model.input_shape[1]+1
next_word=predict_next(model,token,text1,max_seq)
next_word

'fit'

In [56]:
text1='lookes it not like the'
max_seq=model.input_shape[1]+1
next_word=predict_next(model,token,text1,max_seq)
next_word

'gods'

In [36]:
import pickle

In [37]:
model.save('model.h5')
with open('token.pickle','wb') as handle:
    pickle.dump(token,handle,protocol=pickle.HIGHEST_PROTOCOL)